In [1]:
import cv2
from collections import Counter
from deepface import DeepFace
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import base64
import ffmpeg
import time


# Step 1: Split video to frames with OpenCV
def split_video_to_frames(path):
    cap = cv2.VideoCapture(path)
    framerate = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []
    selected_frames = []

    # Calcul du nombre total de frames dans la vidéo
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calcule l'intervalle entre chaque image à extraire
    intervalle = total_frames // 6

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    for i in range(6):
        # Définit le numéro de frame à extraire
        frame_num = intervalle * i
        selected_frames.append(frames[frame_num])

    cap.release()
    return frames,selected_frames, framerate


# in this fucntion all the work is done here so that in spark we launch only dowork()

def dowork(x):
    # Step 2: Take x frames as a list and select a frame as the original
    # video_path = "classrom1.mp4"
    video_path = x
    pp = video_paths[0].split("/")[-1].split(".")[0].split("_")[-1]
    all_frames,selected_frames, original_frame_rate = split_video_to_frames(video_path)
    # selected_frames = all_frames[:10]  # Replace x with the desired number
    original_frame = selected_frames[0]

    # Step 3: Original frame with histogram equalization

    # Convert the image to YUV color space
    yuv_image = cv2.cvtColor(original_frame, cv2.COLOR_BGR2YUV)

    # Apply histogram equalization to the Y channel (luminance)
    yuv_image[:, :, 0] = cv2.equalizeHist(yuv_image[:, :, 0])

    # Convert the image back to BGR color space
    original_frame_equalized = cv2.cvtColor(yuv_image, cv2.COLOR_YUV2BGR)

    # Step 4: Original frame with Bilateral filter (adjust the parameters if needed)
    original_frame_denoised = cv2.bilateralFilter(original_frame, 9, 75, 75)

    # Step 5: Original frame with Canny filter
    original_frame_canny = cv2.Canny(cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY), 50, 150)

    # Step 6: Use DeepFace for face and age detection
    # Step 7: Generate a new video with face and age detection
    output_video_path = f"/SmartEdu_src/result/video___{pp}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, original_frame_rate,
                          (original_frame.shape[1], original_frame.shape[0]))
    # Load pre-trained face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Initialize a counter for emotion frequencies
    emotion_counter = Counter()

    # Assuming 'all_frames' is a list of frames obtained from a video
    for frame in all_frames:

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Process each detected face
        for (x, y, w, h) in faces:
            # Extract the face region
            face_roi = frame[y:y + h, x:x + w]

            # Analyze the face for age and emotion
            result = DeepFace.analyze(face_roi, actions=['age', 'emotion'], enforce_detection=False)

            # Access age and emotion information for each face
            age = result[0]['age']  # Access the first face in the list
            emotion = result[0]['dominant_emotion']  # Access the first face in the list

            # Update the emotion counter
            emotion_counter[emotion] += 1

            # Draw bounding box around the face with age and emotion information
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"Age: {age}, Emotion: {emotion}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 255, 0),
                        2)

        # Save the frame to the output video
        out.write(frame)

    out.release()

    cv2.imwrite(f"/SmartEdu_src/result/original_frame_equalized___{pp}.jpg", original_frame_equalized)
    cv2.imwrite(f"/SmartEdu_src/result/original_frame_denoised___{pp}.jpg", original_frame_denoised)
    cv2.imwrite(f"/SmartEdu_src/result/original_frame_canny___{pp}.jpg", original_frame_canny)
    cv2.imwrite(f"/SmartEdu_src/result/original_frame___{pp}.jpg", original_frame)



    # Print the emotion frequencies
    print("Emotion Frequencies:")
    for emotion, count in emotion_counter.items():
        print(f"{emotion}: {count}")



    return "Emotion Frequencies:", emotion_counter.items()

    

# creating spark application :
# Initialize SparkSession
spark = SparkSession.builder.appName("FaceDetectionApp").getOrCreate()

# Specify the HDFS folder containing video files
# hdfs_video_folder = "hdfs://your-hdfs-path/to/your/video/folder"
hdfs_video_folder = "/SmartEdu_src/videos"

# Get a list of all video files in the HDFS folder
video_paths = [os.path.join(hdfs_video_folder, file) for file in os.listdir(hdfs_video_folder) if
               file.lower().endswith(('.mp4', '.avi', '.mkv'))]

# Distribute video paths using Spark
videos_rdd = spark.sparkContext.parallelize(video_paths, numSlices=len(video_paths) * 3)

# Process videos in parallel using dowork function
results_rdd = videos_rdd.map(lambda x : dowork(x))

# Collect the results
results = results_rdd.collect()

# Stop SparkSession
spark.stop()

# Process the collected results as needed
for result in results:
    print(result)

test = results[0][1]

hello = {}
for i in test:
    hello[i[0]] = i[1]
# hello

hash = video_paths[0].split("/")[-1].split(".")[0].split("_")[-1]

result = {"hash": hash}

emotions = [{"name": key, "count": value} for key, value in hello.items()]

with open(f"/SmartEdu_src/result/original_frame___{hash}.jpg", "rb") as image_file:
    original_frame = base64.b64encode(image_file.read())
with open(f"/SmartEdu_src/result/original_frame_canny___{hash}.jpg", "rb") as image_file:
    canny_frame = base64.b64encode(image_file.read())
with open(f"/SmartEdu_src/result/original_frame_denoised___{hash}.jpg", "rb") as image_file:
    denoised_frame = base64.b64encode(image_file.read())
with open(f"/SmartEdu_src/result/original_frame_equalized___{hash}.jpg", "rb") as image_file:
    equalized_frame = base64.b64encode(image_file.read())

result["processingInfo"] = {
    "emotions": emotions,
    "images": {
        "original": str(original_frame),
        "canny": str(canny_frame),
        "denoised": str(denoised_frame),
        "equalized": str(equalized_frame)
    },
    "processedVideo": f"/source/result/output_video___{hash}.mp4"
}

import requests
import json

json_data = json.dumps(result)

# print(json_data)

# Set the URL of the API endpoint
url = 'http://172.18.0.2:3000/api/videoData'

# Set the headers (optional, depending on API requirements)
headers = {'Content-Type': 'application/json'}

# Send the POST request with JSON payload
response = requests.post(url, data=json_data, headers=headers)

# Check the response status code
if response.status_code == 200:
    print("Data sent successfully!")
else:
    print("Failed to send data. Status code:", response.status_code)
    print("Response content:", response.text)
    
 # ####################################################################

time.sleep(2)

input_stream = ffmpeg.input(f"result/video___{hash}.mp4")
output_stream = ffmpeg.output(input_stream['v'], f"result/output_video___{hash}.mp4", vcodec='h264')

# Run the conversion with error handling, passing the flag
try:
    ffmpeg.run(output_stream)
    print("Video conversion successful!")
except ffmpeg.Error as e:
    print(f"Error: {e}")

2024-02-17 14:21:02.223722: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 14:21:02.910651: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 14:21:02.910797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 14:21:03.063922: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-17 14:21:03.314908: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 14:21:03.318071: I tensorflow/core/platform/cpu_feature_guard.cc:1

24-02-17 14:21:07 - Directory /root/.deepface created
24-02-17 14:21:07 - Directory /root/.deepface/weights created


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/17 14:21:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Action: age:   0%|          | 0/2 [00:00<?, ?it/s]                  (2 + 1) / 3]24-02-17 14:21:25 - age_model_weights.h5 will be downloaded...
Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5

  0%|          | 0.00/539M [00:00<?, ?B/s]
  0%|          | 524k/539M [00:00<08:12, 1.09MB/s]
  0%|          | 1.05M/539M [00:00<04:32, 1.97MB/s]
  0%|          | 2.10M/539M [00:00<02:18, 3.86MB/s]
  1%|          | 4.19M/539M [00:00<01:08, 7.78MB/s]
  1%|          | 6.29M/539M [00:00<00:48, 11.0MB/s]
  2%|▏         | 9.96M/539M [00:01<00:30, 17.5MB/s]
  3%|▎         | 13.6M/539M [00:01<00:23, 22.4MB/s]
  3%|▎         | 17

('Emotion Frequencies:', dict_items([('fear', 52), ('angry', 82), ('neutral', 163), ('happy', 108), ('sad', 48)]))
Data sent successfully!


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Video conversion successful!


frame=  150 fps=127 q=-1.0 Lsize=     820kB time=00:00:04.90 bitrate=1370.5kbits/s speed=4.16x    
video:817kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.290197%
[libx264 @ 0x55d7e917bf80] frame I:1     Avg QP:22.99  size: 32684
[libx264 @ 0x55d7e917bf80] frame P:72    Avg QP:23.96  size:  8710
[libx264 @ 0x55d7e917bf80] frame B:77    Avg QP:26.84  size:  2292
[libx264 @ 0x55d7e917bf80] consecutive B-frames: 21.3% 21.3% 28.0% 29.3%
[libx264 @ 0x55d7e917bf80] mb I  I16..4: 14.7% 76.4%  8.9%
[libx264 @ 0x55d7e917bf80] mb P  I16..4:  1.5%  5.4%  0.8%  P16..4: 39.4% 16.4%  9.1%  0.0%  0.0%    skip:27.4%
[libx264 @ 0x55d7e917bf80] mb B  I16..4:  0.3%  0.7%  0.1%  B16..8: 38.3%  4.2%  0.8%  direct: 1.0%  skip:54.8%  L0:44.0% L1:46.6% BI: 9.4%
[libx264 @ 0x55d7e917bf80] 8x8 transform intra:71.2% inter:71.4%
[libx264 @ 0x55d7e917bf80] coded y,uvDC,uvAC intra: 53.5% 54.6% 18.2% inter: 12.7% 16.0% 4.7%
[libx264 @ 0x55d7e917bf80] i16 v,h,dc,p: 34% 40% 17%  9%
[